In [ ]:
print('Gajraj')

In [ ]:
!pip install duckdb
!pip install plotly
!pip install --upgrade nbformat

In [ ]:
import duckdb
import pandas as pd
import random
import plotly.express as ex
import numpy as np
pd.options.plotting.backend = "plotly"
import plotly.io as pio
import plotly.express as px
pio.renderers.default = "vscode"

In [ ]:
con = duckdb.connect(database=':memory:')

class RETURNTYPE:
    ONE=1
    PANDAS=2
    RECORDS=3

def sql_help(sql,params=RETURNTYPE.PANDAS):
    con.execute(sql)
    if params==RETURNTYPE.ONE:
        return con.fetchone()
    if params==RETURNTYPE.RECORDS:
        return con.fetchall()
    if params==RETURNTYPE.PANDAS:
        return con.fetch_df()                

In [ ]:

df=pd.DataFrame({"x":[random.randint(1,100) for i in range(100)],"y":[random.randint(1,100) for i in range(100)],"z":[random.randint(1,100) for i in range(100)]})
foo=sql_help("select * from df where x < 50;",RETURNTYPE.RECORDS)

In [ ]:
foo.plot.pie(y=df.y)

In [ ]:
import plotly.express as px
fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='y')
fig.show()

##### Pytorch lightning to onnx Eample

In [ ]:
!pip install pytorch-lightning
!pip install  onnxruntime

In [ ]:
%matplotlib inline
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from torchmetrics.functional import accuracy
import pandas as pd

In [ ]:
class SimpleModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(in_features=64, out_features=4)
        self.example_input_array = torch.randn(7, 64)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))
        

In [ ]:
# create the model
model = SimpleModel()
filepath = "model.onnx"
model.to_onnx(filepath, export_params=True)

In [ ]:
import onnxruntime
import numpy as np
ort_session = onnxruntime.InferenceSession(filepath)
input_name = ort_session.get_inputs()[0].name
ort_inputs = {input_name: np.random.randn(7, 64).astype(np.float32)}
ort_outs = ort_session.run(None, ort_inputs)
ort_outs

In [ ]:
ort_outs[0].ndim